In [21]:
!pip install selenium webdriver-manager
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time



In [22]:
!pip install selenium webdriver-manager


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

driver_path = r"D:\Temp\ChromeDriver\chromedriver-win64\chromedriver.exe"  # Update to your path

options = Options()
options.headless = True  # False if you want to see the browser

driver = webdriver.Chrome(
    service=Service(driver_path),
    options=options
)


In [32]:
import time
from bs4 import BeautifulSoup
import pandas as pd

url = "https://remoteok.com/remote-dev-jobs"
driver.get(url)         # Load the page in the browser
time.sleep(5)           # Wait for JavaScript to load content


In [33]:
# Scroll down incrementally to load more jobs
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(10):  # Adjust number of scrolls depending on how many jobs you want
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


In [34]:
soup = BeautifulSoup(driver.page_source, "html.parser")


In [35]:
job_list = []

In [36]:
import re

def extract_salary_and_clean_locations(locations):
    min_salary = None
    max_salary = None

    if not locations:
        return locations, None, None

    potential_salary = locations[-1]
    if "$" in potential_salary:
        locations = locations[:-1]  # Remove salary
        salary_clean = potential_salary.replace("💰", "").replace("$", "").strip()
        print(f"Salary raw: {potential_salary}, cleaned: {salary_clean}")

        try:
            parts = salary_clean.split("-")
            if len(parts) == 2:
                min_match = re.search(r'\d+', parts[0])
                max_match = re.search(r'\d+', parts[1])
                if min_match and max_match:
                    min_salary = int(min_match.group()) * 1000
                    max_salary = int(max_match.group()) * 1000
            elif len(parts) == 1:
                match = re.search(r'\d+', parts[0])
                if match:
                    min_salary = int(match.group()) * 1000
        except Exception as e:
            print(f"Failed to parse salary: {e}")

    return locations, min_salary, max_salary


In [37]:
i = 0
for job in soup.find_all("tr", class_="job"):
    print(f"Processing job {i}")
    i += 1
    # Get the job title
    title_tag = job.find("h2", itemprop="title")
    title = title_tag.text.strip() if title_tag else None

    # Get the company name
    company_tag = job.find("h3", itemprop="name")
    company = company_tag.text.strip() if company_tag else None

    # Get job link (relative path, needs prefix)
    link_tag = job.find("a", class_="preventLink")
    relative_link = link_tag.get("href") if link_tag else None
    link = f"https://remoteok.com{relative_link}" if relative_link else None

    # Get all locations
    location_tags = job.find_all("div", class_="location")
    locations = [loc.text.strip() for loc in location_tags]

    
    locations, min_salary, max_salary = extract_salary_and_clean_locations(locations)

    # Optional: check if verified
    verified = job.find("span", class_="verified") is not None

    tags_td = job.find('td', class_='tags')
    tag_elements = tags_td.find_all('h3') if tags_td else []
    tags = [tag.text.strip() for tag in tag_elements]
    # print(f"company: {company}", f"Tags: {tags}")

    time = job.find('td', class_='time')
    time = time.text.strip() if time else None
    # print(company,f"Time: {time}")
    # print(time)

    # Add to list
    job_list.append({
        "title": title,
        "company": company,
        "link": link,
        "locations": ", ".join(locations),
        "min salary": min_salary,
        "max salary": max_salary,
        "verified": verified,
        "tags": tags,
        "time": time
    })
    

print(job_list[0])

Processing job 0
Salary raw: 💰 $20k - $40k, cleaned: 20k - 40k
Processing job 1
Salary raw: 💰 $140k - $200k, cleaned: 140k - 200k
Processing job 2
Salary raw: 💰 $20k - $40k, cleaned: 20k - 40k
Processing job 3
Salary raw: 💰 $30k - $40k, cleaned: 30k - 40k
Processing job 4
Salary raw: 💰 $30k - $40k, cleaned: 30k - 40k
Processing job 5
Salary raw: 💰 $120k - $140k, cleaned: 120k - 140k
Processing job 6
Salary raw: 💰 $50k - $90k, cleaned: 50k - 90k
Processing job 7
Salary raw: 💰 $50k - $60k, cleaned: 50k - 60k
Processing job 8
Salary raw: 💰 $30k - $40k, cleaned: 30k - 40k
Processing job 9
Salary raw: 💰 $40k - $60k, cleaned: 40k - 60k
Processing job 10
Salary raw: 💰 $60k - $80k, cleaned: 60k - 80k
Processing job 11
Salary raw: 💰 $80k - $100k, cleaned: 80k - 100k
Processing job 12
Salary raw: 💰 $60k - $140k, cleaned: 60k - 140k
Processing job 13
Salary raw: 💰 $60k - $120k, cleaned: 60k - 120k
Processing job 14
Salary raw: 💰 $60k - $90k, cleaned: 60k - 90k
Processing job 15
Salary raw: 💰 $60k

In [38]:
len(job_list)

70

In [39]:
len(job_list)
job_list

[{'title': 'WordPress Plugins & Themes developer',
  'company': 'Melapress',
  'link': 'https://remoteok.com/remote-jobs/remote-wordpress-plugins-themes-developer-melapress-1093690',
  'locations': '🇪🇺 Europe, 🇬🇧 United Kingdom, 🇹🇷 Turkey',
  'min salary': 20000,
  'max salary': 40000,
  'verified': True,
  'tags': ['DevOps',
   'Wordpress',
   'PHP',
   'JavaScript',
   'Full Time',
   'Ecommerce',
   'API',
   'English'],
  'time': '1d'},
 {'title': 'Software Engineer Data Infrastructure & Acquisition',
  'company': 'Speechify',
  'link': 'https://remoteok.com/remote-jobs/remote-software-engineer-data-infrastructure-acquisition-speechify-1093699',
  'locations': '🌏 Worldwide',
  'min salary': 140000,
  'max salary': 200000,
  'verified': True,
  'tags': [],
  'time': '55m'},
 {'title': 'Content Producer Technical Content Engineer Junior Dev',
  'company': 'Private.Ki',
  'link': 'https://remoteok.com/remote-jobs/remote-content-producer-technical-content-engineer-junior-dev-private-ki

In [40]:
job_list_pd = pd.DataFrame(job_list)
job_list_pd

,title,company,link,locations,min salary,max salary,verified,tags,time
0,WordPress Plugins & Themes developer,Melapress,https://remoteok.com/remote-jobs/remote-wordpr...,"🇪🇺 Europe, 🇬🇧 United Kingdom, 🇹🇷 Turkey",20000.0,40000.0,True,"[DevOps, Wordpress, PHP, JavaScript, Full Time...",1d
1,Software Engineer Data Infrastructure & Acquis...,Speechify,https://remoteok.com/remote-jobs/remote-softwa...,🌏 Worldwide,140000.0,200000.0,True,[],55m
2,Content Producer Technical Content Engineer Ju...,Private.Ki,https://remoteok.com/remote-jobs/remote-conten...,🌏 Worldwide,20000.0,40000.0,False,"[Other, Writer, Writing, Marketing]",2d
3,Senior Full Stack Engineer,AMK Solutions,https://remoteok.com/remote-jobs/remote-senior...,🌏 Worldwide,30000.0,40000.0,False,[],5d
4,Backend Developer @ lobstr.io Python Scraping ...,lobstr.io,https://remoteok.com/remote-jobs/remote-backen...,🌏 Worldwide,30000.0,40000.0,False,"[Python, Backend, Redis, Remote, API, Tech Lea...",12d
...,...,...,...,...,...,...,...,...,...
65,Developer,VIVA,https://remoteok.com/remote-jobs/remote-develo...,🌏 Worldwide,60000.0,100000.0,False,"[JavaScript, Backend, Front End, Ruby, Tailwind]",1yr
66,Senior Developer,CoLab Software,https://remoteok.com/remote-jobs/remote-senior...,🇨🇦 Canada,100000.0,120000.0,True,[Senior],1yr
67,Senior Product Engineer,Plain,https://remoteok.com/remote-jobs/remote-senior...,🇪🇺 Europe,90000.0,140000.0,True,"[Backend, Engineer]",1yr
68,Senior Software Developer Payroll & Compensation,Jane Software,https://remoteok.com/remote-jobs/remote-senior...,🇨🇦 Canada,70000.0,110000.0,False,"[Developer, Software, Payroll, Design, Python,...",1yr


In [42]:
job_list_pd.to_csv("remote_jobs.csv", index=False, encoding='utf-8-sig')